Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Setup AML Workspace
---

This notebook walks you through all the necessary steps to configure your environment for this solution accelerator including:

1. Connecting to your workspace 
2. Create a config.json (this can be skipped if running on an AML compute instance)
3. Create a compute cluster


## 1.0 Connect to workspace

Connect this solution accelerator to your AML workspace. This step isn't necessary if you're using a Notebook VM.

The following cell allows you to specify your workspace parameters. This cell uses the python method os.getenv to read values from environment variables which is useful for automation. If no environment variable exists, the parameters will be set to the specified default values.

In [ ]:
import os

subscription_id = os.getenv("SUBSCRIPTION_ID", default="<my-subscription-id>")
resource_group = os.getenv("RESOURCE_GROUP", default="<my-resource-group>")
workspace_name = os.getenv("WORKSPACE_NAME", default="<my-workspace-name>")
workspace_region = os.getenv("WORKSPACE_REGION", default="westus2")

In [ ]:
from azureml.core import Workspace

try:
    ws = Workspace(subscription_id=subscription_id, 
                   resource_group=resource_group, 
                   workspace_name=workspace_name)
    print("Workspace configuration succeeded. Skip the workspace creation steps below")
except:
    print("Workspace does not exist. Creating workspace")
    ws = Workspace.create(name=workspace_name, subscription_id=subscription_id, resource_group=resource_group,
                            location=workspace_region, create_resource_group=True, sku='enterprise', exist_ok=True)

In [ ]:
ws.get_details()

## 2.0 Write config file
Write the details of the workspace to a config.json file:

In [ ]:
ws.write_config()

## 3.0 Create compute cluster

You will need a compute cluster for training and batch forecasting.
This is a one-time set up so you won't need to re-run this in future notebooks, but you'll need to use the same cluster name to reference this cluster in following notebooks.

In [ ]:
# Choose a name for your CPU cluster
cpu_cluster_name = 'cpucluster'

We create a STANDARD_D13_V2 compute cluster. D-series VMs are used for tasks that require higher compute power and temporary disk performance. This [page](https://docs.microsoft.com/azure/cloud-services/cloud-services-sizes-specs) will give you more information on VM sizes to help you decide which will best fit your use case.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found an existing cluster, using it instead.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D13_V2',
                                                           min_nodes=0,
                                                           max_nodes=20)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    cpu_cluster.wait_for_completion(show_output=True)

## Next Steps

Now that the AML Workspace its configured, it's time to create the sample datasets. Follow the steps in [01_Data_Preparation.ipynb](01_Data_Preparation.ipynb) for that.